# Non-negative Matrix Factorization (CMF)
## Init

In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
from functools import partial

import plotly.express as px


# import implicit # Matrix Factorization

from sklearn.decomposition import NMF
from sklearn.model_selection import KFold
from cmfrec import CMF_implicit

In [2]:
import sys
sys.path.append("../")
from src.data_preprocessing import TrainTestGenerator
from src.evaluator import Evaluator

In [3]:
show_ploty = False

In [4]:
data_dir = "../data/"
data_generator = TrainTestGenerator(data_dir)

## Model

In [5]:
# Model wrapper

class CMF_recommender:
    def __init__(self, k=50):
        self.model = CMF_implicit(
            # verbose=False,
            # method="als",
            nonneg=True,
            k=k,
            random_state=1,
            # lambda_=1e+1
        )

    def fit(self, data: pd.DataFrame):
        data = data.copy()
        data = data.rename(columns={
            "userID": "UserId",
            "artistID": "ItemId",
            "weight": "Rating"
        })
        self.model.fit(data)

    def recommend(self, user_id, n):
        recommendations = self.model.topN(user_id, n=n)
        return recommendations

## Evaluation

In [6]:
# Evaluator (forward chaining)

evaluator = Evaluator(CMF_recommender, data_generator)
evaluator.evaluate()

evaluator.save_results("../results/cmf_nonneg_ranks.csv", "../results/cmf_nonneg_times.csv")

In [7]:
# Hit Rate

evaluator.get_hit_rates()

,cases,5,10,25,50,500
2008,4556,0.012511,0.017779,0.037972,0.072212,0.300922
2009,4687,0.008961,0.013868,0.026029,0.049712,0.259441
2010,6133,0.015164,0.024458,0.045981,0.070276,0.295940
2011,1129,0.018601,0.035430,0.062888,0.098317,0.365810


In [8]:
# Mean Reciprocal Rank

evaluator.get_mrr()

,cases,mrr
2008,2608,0.018926
2009,3086,0.012757
2010,4306,0.015186
2011,878,0.023314


In [9]:
# Times

evaluator.get_times()

,count,mean,std,min,25%,50%,75%,max
task,,,,,,,,
model_fit,4.0,0.739946,0.189148,0.594614,0.601979,0.682809,0.820776,0.999550
model_init,4.0,0.000024,0.000011,0.000016,0.000017,0.000020,0.000027,0.000040
recommend_user,2622.0,0.000765,0.000461,0.000370,0.000576,0.000727,0.000865,0.011594


In [10]:
evaluator.get_fit_per_year_times()

,tag,time
task,,
model_fit,model_fit_2008,0.594614
model_fit,model_fit_2009,0.604434
model_fit,model_fit_2010,0.761184
model_fit,model_fit_2011,0.999550
